In [2]:
# import libraries 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os, re, time, math, tqdm, itertools
import matplotlib.pyplot as plt
import seaborn as sns #資料視覺化
import plotly.express as px
import plotly.offline as pyo
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.neural_network import MLPClassifier
import keras
from keras.layers import Conv2D, Conv1D, MaxPooling2D, MaxPooling1D, Flatten, BatchNormalization, Dense
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.callbacks import CSVLogger, ModelCheckpoint

2022-11-11 11:15:53.844156: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-11 11:15:54.013328: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-11 11:15:54.474426: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/ne6101157/anaconda3/env/trackformer/lib
2022-11-11 11:15:54.474548: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinf

In [3]:
def load_data(Input_csv, label):
    df=pd.read_csv(Input_csv, index_col=None, low_memory=False)

    print(Input_csv,"資料總數: ",df.shape) #檢查數據的大小 
    df['Label'].value_counts()
    df=df[df['Label'].isin([label])]
    print(label,"選擇資料總數: ",df.shape)
    df['Label'].value_counts()
    print("++++++++++++++++++++++++++++++++++++++++++++++++++")
    df_list.append(df)
    return "Done!"

In [4]:
df_list=[]
load_data('./CIC-IDS2018/02-22-2018.csv', 'Benign') #Benign 1048213
load_data('./CIC-IDS2018/02-14-2018.csv', 'Benign') #Benign 667626
load_data('./CIC-IDS2018/02-15-2018.csv', 'DoS attacks-GoldenEye') #DoS attacks-Gloden Eye 41508
load_data('./CIC-IDS2018/02-16-2018.csv', 'DoS attacks-SlowHTTPTest') #DoS attacks-SlowHTTPTest 139890
load_data('./CIC-IDS2018/02-15-2018.csv', 'DoS attacks-Slowloris') #DoS attacks-Slowloris 10990
load_data('./CIC-IDS2018/02-14-2018.csv', 'FTP-BruteForce') #FTP-Brute Force 193360
load_data('./CIC-IDS2018/02-20-2018.csv', 'DDoS attacks-LOIC-HTTP') #DDoS attacks-LOIC-HTTP 576191
load_data('./CIC-IDS2018/03-02-2018.csv', 'Bot') #Bot 286191
df = pd.concat(df_list, axis=0, ignore_index=True)

./CIC-IDS2018/02-22-2018.csv 資料總數:  (1048575, 80)
Benign 選擇資料總數:  (1048213, 80)
++++++++++++++++++++++++++++++++++++++++++++++++++
./CIC-IDS2018/02-14-2018.csv 資料總數:  (1048575, 80)
Benign 選擇資料總數:  (667626, 80)
++++++++++++++++++++++++++++++++++++++++++++++++++
./CIC-IDS2018/02-15-2018.csv 資料總數:  (1048575, 80)
DoS attacks-GoldenEye 選擇資料總數:  (41508, 80)
++++++++++++++++++++++++++++++++++++++++++++++++++
./CIC-IDS2018/02-16-2018.csv 資料總數:  (1048575, 80)
DoS attacks-SlowHTTPTest 選擇資料總數:  (139890, 80)
++++++++++++++++++++++++++++++++++++++++++++++++++
./CIC-IDS2018/02-15-2018.csv 資料總數:  (1048575, 80)
DoS attacks-Slowloris 選擇資料總數:  (10990, 80)
++++++++++++++++++++++++++++++++++++++++++++++++++
./CIC-IDS2018/02-14-2018.csv 資料總數:  (1048575, 80)
FTP-BruteForce 選擇資料總數:  (193360, 80)
++++++++++++++++++++++++++++++++++++++++++++++++++
./CIC-IDS2018/02-20-2018.csv 資料總數:  (7948748, 84)
DDoS attacks-LOIC-HTTP 選擇資料總數:  (576191, 84)
++++++++++++++++++++++++++++++++++++++++++++++++++
./CIC-IDS2018/03-02

In [5]:
df.shape #檢查數據的大小 
print('Rows 個數(Sample): %d' %  df.shape[0])
print('Columns 個數(Feature): %d' %  df.shape[1])
df['Label'].value_counts()

Rows 個數(Sample): 2963969
Columns 個數(Feature): 84


Benign                      1715839
DDoS attacks-LOIC-HTTP       576191
Bot                          286191
FTP-BruteForce               193360
DoS attacks-SlowHTTPTest     139890
DoS attacks-GoldenEye         41508
DoS attacks-Slowloris         10990
Name: Label, dtype: int64

In [6]:
print('總共的Columns數: %d' % len(df.columns))

總共的Columns數: 84


In [7]:
df.columns #檢查資料的columns有哪些
df = df.drop(['Flow ID', 'Src IP', 'Src Port', 'Dst IP'], axis=1)

In [8]:
chage_dtype={'Dst Port': 'int', 'Protocol': 'int', 'Timestamp': 'object', 'Flow Duration': 'int', 'Tot Fwd Pkts': 'int', 'Tot Bwd Pkts': 'int', 'TotLen Fwd Pkts': 'int', 'TotLen Bwd Pkts': 'int', 'Fwd Pkt Len Max': 'int', 'Fwd Pkt Len Min': 'int', 'Fwd Pkt Len Mean': 'float', 'Fwd Pkt Len Std': 'float', 'Bwd Pkt Len Max': 'int', 'Bwd Pkt Len Min': 'int', 'Bwd Pkt Len Mean': 'float', 'Bwd Pkt Len Std': 'float', 'Flow Byts/s': 'float', 'Flow Pkts/s': 'float', 'Flow IAT Mean': 'float', 'Flow IAT Std': 'float', 'Flow IAT Max': 'int', 'Flow IAT Min': 'int', 'Fwd IAT Tot': 'int', 'Fwd IAT Mean': 'float', 'Fwd IAT Std': 'float', 'Fwd IAT Max': 'int', 'Fwd IAT Min': 'int', 'Bwd IAT Tot': 'int', 'Bwd IAT Mean': 'float', 'Bwd IAT Std': 'float', 'Bwd IAT Max': 'int', 'Bwd IAT Min': 'int', 'Fwd PSH Flags': 'int', 'Bwd PSH Flags': 'int', 'Fwd URG Flags': 'int', 'Bwd URG Flags': 'int', 'Fwd Header Len': 'int', 'Bwd Header Len': 'int', 'Fwd Pkts/s': 'float', 'Bwd Pkts/s': 'float', 'Pkt Len Min': 'int', 'Pkt Len Max': 'int', 'Pkt Len Mean': 'float', 'Pkt Len Std': 'float', 'Pkt Len Var': 'float', 'FIN Flag Cnt': 'int', 'SYN Flag Cnt': 'int', 'RST Flag Cnt': 'int', 'PSH Flag Cnt': 'int', 'ACK Flag Cnt': 'int', 'URG Flag Cnt': 'int', 'CWE Flag Count': 'int', 'ECE Flag Cnt': 'int', 'Down/Up Ratio': 'int', 'Pkt Size Avg': 'float', 'Fwd Seg Size Avg': 'float', 'Bwd Seg Size Avg': 'float', 'Fwd Byts/b Avg': 'int', 'Fwd Pkts/b Avg': 'int', 'Fwd Blk Rate Avg': 'int', 'Bwd Byts/b Avg': 'int', 'Bwd Pkts/b Avg': 'int', 'Bwd Blk Rate Avg': 'int', 'Subflow Fwd Pkts': 'int', 'Subflow Fwd Byts': 'int', 'Subflow Bwd Pkts': 'int', 'Subflow Bwd Byts': 'int', 'Init Fwd Win Byts': 'int', 'Init Bwd Win Byts': 'int', 'Fwd Act Data Pkts': 'int', 'Fwd Seg Size Min': 'int', 'Active Mean': 'float', 'Active Std': 'float', 'Active Max': 'int', 'Active Min': 'int', 'Idle Mean': 'float', 'Idle Std': 'float', 'Idle Max': 'int', 'Idle Min': 'int', 'Label': 'object'}

In [9]:
df=df.astype(chage_dtype)

In [10]:
df.info() #顯示詳細資料

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2963969 entries, 0 to 2963968
Data columns (total 80 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Dst Port           int64  
 1   Protocol           int64  
 2   Timestamp          object 
 3   Flow Duration      int64  
 4   Tot Fwd Pkts       int64  
 5   Tot Bwd Pkts       int64  
 6   TotLen Fwd Pkts    int64  
 7   TotLen Bwd Pkts    int64  
 8   Fwd Pkt Len Max    int64  
 9   Fwd Pkt Len Min    int64  
 10  Fwd Pkt Len Mean   float64
 11  Fwd Pkt Len Std    float64
 12  Bwd Pkt Len Max    int64  
 13  Bwd Pkt Len Min    int64  
 14  Bwd Pkt Len Mean   float64
 15  Bwd Pkt Len Std    float64
 16  Flow Byts/s        float64
 17  Flow Pkts/s        float64
 18  Flow IAT Mean      float64
 19  Flow IAT Std       float64
 20  Flow IAT Max       int64  
 21  Flow IAT Min       int64  
 22  Fwd IAT Tot        int64  
 23  Fwd IAT Mean       float64
 24  Fwd IAT Std        float64
 25  Fwd IAT Max       

## **數據前處理**
數據預處理在數據科學過程中起著重要作用，因為數據可能不完全乾淨，並且可能包含缺失值或空值。
在這一步中，我們正在進行一些預處理步驟，如果我們的數據中有任何空值或缺失值，這將有助於我們。

In [11]:
#采用isna()函數以檢測 DataFrame 中的缺失值。
df.isna().sum().to_numpy()
#我們需要從我們的數據中刪除"此列"(不是刪除此特徵)，以便我們的數據可以被清理。

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0, 5846,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0])

In [12]:
# 刪除空列或缺失列
df=df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2958123 entries, 0 to 2963968
Data columns (total 80 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Dst Port           int64  
 1   Protocol           int64  
 2   Timestamp          object 
 3   Flow Duration      int64  
 4   Tot Fwd Pkts       int64  
 5   Tot Bwd Pkts       int64  
 6   TotLen Fwd Pkts    int64  
 7   TotLen Bwd Pkts    int64  
 8   Fwd Pkt Len Max    int64  
 9   Fwd Pkt Len Min    int64  
 10  Fwd Pkt Len Mean   float64
 11  Fwd Pkt Len Std    float64
 12  Bwd Pkt Len Max    int64  
 13  Bwd Pkt Len Min    int64  
 14  Bwd Pkt Len Mean   float64
 15  Bwd Pkt Len Std    float64
 16  Flow Byts/s        float64
 17  Flow Pkts/s        float64
 18  Flow IAT Mean      float64
 19  Flow IAT Std       float64
 20  Flow IAT Max       int64  
 21  Flow IAT Min       int64  
 22  Fwd IAT Tot        int64  
 23  Fwd IAT Mean       float64
 24  Fwd IAT Std        float64
 25  Fwd IAT Max       

## **label Encoding**
數據中的Label Features包含 3 個Labels，分別是 Benign、BruteForceFTP 和 BruteForceSSH
所有這些都是字符串格式。
對於我們的神經網絡，我們需要將它們轉換為數字，以便我們的 NN 可以理解它們的表示。

In [13]:
# #labelencoder是將文字mapping到一個數字，類似的有one hot encoding
label_encoder = LabelEncoder()
df['Label']= label_encoder.fit_transform(df['Label'])
df['Label'].unique()
# for row in df:
#     if row['Label']=='Benign':
#         row['Label'] = 0
#     else: row['Label'] = 1

array([0, 3, 4, 5, 6, 2, 1])

In [14]:
df['Label'].value_counts()

0    1709999
2     576191
1     286191
6     193354
4     139890
3      41508
5      10990
Name: Label, dtype: int64

In [15]:
# 為CNN塑造數據，必須遵循以下步驟
#1.Seperate the data of each of the labels (分離每個標籤的數據)
#2.Create a numerical matrix representation of labels(創建標籤的數字矩陣表示)
#3.Apply resampling on data so that can make the distribution equal for all labels(對數據應用重採樣，以使所有標籤的分佈均等)
#4.Create X (predictor) and Y (target) variables(創建 X（預測）和 Y（目標）變量)
#5.Split the data into train and test sets(將數據拆分為訓練集和測試集)
#6.Make data multi-dimensional for CNN(為 CNN 製作多維數據)
#7.Apply CNN on data(將 CNN 應用於數據)

In [16]:
# make 3 seperate datasets for 3 feature labels

data_0 = df[df['Label'] == 0]
data_1 = df[df['Label'] == 1]
data_2 = df[df['Label'] == 2]
data_3 = df[df['Label'] == 3]
data_4 = df[df['Label'] == 4]
data_5 = df[df['Label'] == 5]
data_6 = df[df['Label'] == 6]
# data_7 = df[df['Label'] == 7]
# data_8 = df[df['Label'] == 8]
# data_9 = df[df['Label'] == 9]
# data_10 = df[df['Label'] == 10]

# make benign feature
y_0 = np.zeros(data_0.shape[0])
y_benign = pd.DataFrame(y_0)

# make botnet feature
malicious = pd.concat([data_1, data_2, data_3, data_4, data_5, data_6])
y_1 = np.ones(malicious.shape[0])
y_malicious = pd.DataFrame(y_1)

# # make ddos_HOIC feature
# y_2 = np.full(data_2.shape[0],2)
# y_ddos_HOIC = pd.DataFrame(y_2)

# # make ddos_LOIC_http feature
# y_3 = np.full(data_3.shape[0], 3)
# y_ddos_LOIC_http = pd.DataFrame(y_3)

# # make dos_goldeneyes feature
# y_4 = np.full(data_4.shape[0], 4)
# y_dos_goldeneyes = pd.DataFrame(y_4)

# # make dos_hlk feature
# y_5 = np.full(data_5.shape[0], 5)
# y_dos_hulk = pd.DataFrame(y_5)

# # make dos_slow_http feature
# y_6 = np.full(data_6.shape[0], 6)
# y_dos_slow_http = pd.DataFrame(y_6)

# # make dos_slowloris feature
# y_7 = np.full(data_7.shape[0], 7)
# y_dos_slowloris = pd.DataFrame(y_7)

# # make brute_FTP feature
# y_8 = np.full(data_8.shape[0], 8)
# y_brute_FTP = pd.DataFrame(y_8)

# # make Infileration feature
# y_9 = np.full(data_9.shape[0], 9)
# y_Infileration = pd.DataFrame(y_9)

# # make brute_SSH feature
# y_10 = np.full(data_10.shape[0], 10)
# y_brute_SSH = pd.DataFrame(y_10)

# merging the original dataframe
X = pd.concat([data_0,malicious])
y = pd.concat([y_benign, y_malicious])

In [17]:
print(X.shape)
print(y.shape)

(2958123, 80)
(2958123, 1)


In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [19]:
X_train = X_train.drop(columns = ["Timestamp", "Protocol","PSH Flag Cnt","Init Fwd Win Byts","Flow Byts/s","Flow Pkts/s", "Label"], axis=1)
X_test = X_test.drop(columns = ["Timestamp", "Protocol","PSH Flag Cnt","Init Fwd Win Byts","Flow Byts/s","Flow Pkts/s", "Label"], axis=1)

In [20]:
y_train = to_categorical(y_train, num_classes=2)
y_test = to_categorical(y_test, num_classes=2)

In [21]:
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
# y_train = y_train.to_numpy()
# y_test = y_test.to_numpy()
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1981942, 73)
(976181, 73)
(1981942, 2)
(976181, 2)


In [31]:
# reshape the data for CNN
X_train = X_train.reshape(len(X_train), X_train.shape[1])
X_test = X_test.reshape(len(X_test), X_test.shape[1])
X_train.shape, X_test.shape

((1981942, 73), (976181, 73))

## **CNN model**

In [23]:
# making the deep learning function
def model():
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=6, activation='relu', 
                    padding='same', input_shape=(73, 1)))
    model.add(BatchNormalization())
    
    # adding a pooling layer
    model.add(MaxPooling1D(pool_size=(3), strides=2, padding='same'))
    
    model.add(Conv1D(filters=64, kernel_size=6, activation='relu', 
                    padding='same', input_shape=(73, 1)))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=(3), strides=2, padding='same'))
    
    model.add(Conv1D(filters=64, kernel_size=6, activation='relu', 
                    padding='same', input_shape=(73, 1)))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=(3), strides=2, padding='same'))
    
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(2, activation='softmax'))
    
    opt = keras.optimizers.Adam(learning_rate=0.01)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

In [24]:
model = model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 73, 64)            448       
                                                                 
 batch_normalization (BatchN  (None, 73, 64)           256       
 ormalization)                                                   
                                                                 
 max_pooling1d (MaxPooling1D  (None, 37, 64)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 37, 64)            24640     
                                                                 
 batch_normalization_1 (Batc  (None, 37, 64)           256       
 hNormalization)                                                 
                                                        

2022-11-11 11:18:04.150625: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-11 11:18:04.200975: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/ne6101157/anaconda3/env/trackformer/lib
2022-11-11 11:18:04.200994: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-11-11 11:18:04.202527: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow bin

In [25]:
# logger = CSVLogger('logs.csv', append=True)
# his = model.fit(X_train, y_train, epochs=10, batch_size=128, 
# validation_data=(X_test, y_test), callbacks=[logger])

In [26]:
# # check the model performance on test data
# scores = model.evaluate(X_test, y_test)
# print("%s: %.2f%%" % (model.metrics_names[1], scores[1] * 100))

In [27]:
# # check history of model
# history = his.history
# history.keys()

In [28]:
# epochs = range(1, len(history['loss']) + 1)
# acc = history['accuracy']
# loss = history['loss']
# val_acc = history['val_accuracy']
# val_loss = history['val_loss']

# # visualize training and val accuracy
# plt.figure(figsize=(10, 5))
# plt.title('Training and Validation Accuracy (CNN)')
# plt.xlabel('Epochs')
# plt.ylabel('Accuracy')
# plt.plot(epochs, acc, label='accuracy')
# plt.plot(epochs, val_acc, label='val_acc')
# plt.legend()

# # visualize train and val loss
# plt.figure(figsize=(10, 5))
# plt.title('Training and Validation Loss(CNN)')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.plot(epochs, loss, label='loss', color='g')
# plt.plot(epochs, val_loss, label='val_loss', color='r')
# plt.legend()

In [29]:
# model.save('cic-ids.h5')

## **XGBoost**

In [32]:
import xgboost
from xgboost import XGBClassifier
# define the datasets to evaluate each iteration
evalset = [(X_train, y_train), (X_test,y_test)]

# 建立 XGBClassifier 模型
xgboostModel = XGBClassifier(n_estimators=100, learning_rate= 0.3)
# 使用訓練資料訓練模型
xgboostModel.fit(X_train, y_train, eval_metric='logloss', eval_set=evalset)

# evaluate performance
yhat = xgboostModel.predict(X_test)
# # 使用訓練資料預測分類
# predicted = xgboostModel.predict(X_train)

results = xgboostModel.evals_result()

In [33]:
# 預測成功的比例
print('訓練集: ',xgboostModel.score(X_train,y_train))
print('測試集: ',xgboostModel.score(X_test,y_test))

訓練集:  0.9996947438421507
測試集:  0.999602532727025


In [34]:
xgboostModel.save_model('cic_xgboost.h5')